In [ ]:
# instalacja i import bibliotek

#!pip install langchain_community
#!pip install chromadb

import os
from pathlib import Path
from langchain_core.documents import Document
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#zmienne srodowiskowe
model_name = "speakleash/Bielik-11B-v2.3-Instruct:Q4_K_M" #"SpeakLeash/bielik-11b-v2.3-instruct:Q4_k_M" #"mistral"
embedding_model_name = "speakleash/Bielik-11B-v2.3-Instruct:Q4_K_M" #"ipipan/silver-retriever-base-v1.1" #"mistral"
#folder_path = "/content/dokumenty_rag_demo"
folder_path = "F:\BusinessAI\Projekt koncowy\Materialy\data4ragPL"
hello_msg = "Witaj, z tej strony techniczny asysten AI firmy vet-eye. W czym mogę Ci pomóc?"

In [ ]:
#funkcja pozwalająca na wczytanie plików - intersują nas pliki tekstowe i pdf
def load_documents(folder_path: str):
    loader_txt = DirectoryLoader(
        folder_path,
        glob="**/*.txt",
        loader_cls=TextLoader,
        loader_kwargs={"encoding": "utf-8"},
        recursive=True
    )
    loader_pdf = DirectoryLoader(
        folder_path,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
        recursive=True
    )
    return loader_txt.load() + loader_pdf.load()

In [ ]:
#funkcja dzieląca dokumenty na chunki
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(documents)

#funkcja zwracająca embeddingi i baze wektorową
def build_vectorstore(documents):
    embeddings = OllamaEmbeddings(model=embedding_model_name)
    return Chroma.from_documents(documents, embeddings)

#funkcja przeszukująca bazę wektorową
def get_retriever(vectorstore):
    return vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
# budujemy RAG'a
def build_rag_chain(retriever, llm):
    # Pipeline: input --> query --> dokumenty --> prompt --> LLM --> output --> add info about AI gen content

    #obieranie dokumentów
    retrieve_docs = RunnableLambda(lambda query: {"docs": retriever.get_relevant_documents(query), "question": query})

    #Łączenie dokumentów i pytania w prompt
    def format_prompt(inputs):
        docs = "\n\n".join([doc.page_content for doc in inputs["docs"]])
        prompt = f"### Zadanie: \n Jesteś asystentem technicznym, udzielającym wsparcia pierwszego kontaktu dla produktów vet-eye. Odpowiadaj na zapytanie użytkownika, korzystając z dostarczonego kontekstu, włączając cytowania w formacie [source_id] tylko wtedy, gdy w kontekście znajduje się znacznik <source_id>. \n\n### Wytyczne:\n- Jeśli nie znasz odpowiedzi, wyraźnie to zaznacz.\n- Jeśli nie jesteś pewien, poproś użytkownika o doprecyzowanie.\n- Odpowiadaj w tym samym języku, w którym sformułowane zostało zapytanie użytkownika.\n- Jeśli kontekst jest nieczytelny lub słabej jakości, poinformuj o tym użytkownika i udziel możliwie najlepszej odpowiedzi.\n- Jeśli odpowiedź nie znajduje się w kontekście, ale posiadasz wiedzę na ten temat, poinformuj o tym użytkownika i udziel odpowiedzi w oparciu o własne zrozumienie.\n- Cytowania w formacie [source_id] umieszczaj wyłącznie, jeśli w kontekście występuje znacznik <source_id>.\n- Nie dodawaj cytowań, jeśli nie ma podanego znacznika <source_id> w kontekście.\n- Nie używaj znaczników XML w odpowiedzi.\n- Cytowania powinny być zwięzłe i odnosić się bezpośrednio do podanych informacji.\n\n### Przykład cytowania:\nJeśli użytkownik zapyta o konkretny temat, a informacja znajduje się w \"whitepaper.pdf\" z podanym <source_id>, \nodpowiedź powinna wyglądać następująco:\n\n„Zgodnie z badaniem, proponowana metoda zwiększa wydajność o 20% [whitepaper.pdf].”\nJeśli znacznik <source_id> nie został podany, nie należy dołączać cytowania.\n\n### Wynik:\nUdziel jasnej i konkretnej odpowiedzi na zapytanie użytkownika, wstawiając cytowania w formacie [source_id] tylko wtedy, gdy w kontekście znajduje się znacznik <source_id>. Na koniec wypowiedzi dodaj informację, że odpowiedź została wygenerowana przez sztuczną inteligencję. Pytanie: {inputs['question']} Kontekst: \n\n{docs}\n\n"
        #prompt = f"Na podstawie poniższych dokumentów, odpowiedz na pytanie:\n\n{docs}\n\nPytanie: {inputs['question']}"
        return prompt

    prompt_formatter = RunnableLambda(format_prompt)

    #llm z parserem odpowiedzi
    llm_chain = llm | StrOutputParser()

    add_signature = RunnableLambda(
        lambda text: text + "\n\nTekst został wygenerowany przy użyciu AI."
    )
    #zwracamy caly pipline
    return retrieve_docs | prompt_formatter | llm_chain | add_signature

In [ ]:
#składamy wszystko w całość
def main():
    print("uruchamianie...")
    raw_docs = load_documents(folder_path)
    split_docs = split_documents(raw_docs)
    vectorstore = build_vectorstore(split_docs)
    retriever = get_retriever(vectorstore)
    llm = Ollama(model=model_name)
    rag_chain = build_rag_chain(retriever, llm)

    print(hello_msg)
    while True:
        query = input("> ")
        if query.lower() in ("exit", "quit"):
            print("Żegnaj!")
            break

        response = rag_chain.invoke(query)
        print(response)
        print("—" * 40)

if __name__ == "__main__":
    main()